#Set Directories

In [166]:
import os

# Directories - change based on where your project is saved
project_dir = "/content/drive/MyDrive/CS2822_Final_Project"

# CHANGE THESE
normal_model_name = "entire_dataset_resnet18.a1-e=12-lr=0.01_limit=20.pt"
bugged_model_name = "o2o_easy_resnet18.a1-e=2-lr=0.01_limit=20.pt"
concept_instance_name = "CRAFT_Corgi_Snow_320_im_8_concepts_o2o_easy_resnet18.a1-e=12-lr=0.01_limit=20.pt"

In [101]:
models_dir = os.path.join(project_dir, "Models")
datasets_dir = os.path.join(project_dir, "Datasets")
RESULTS_DIR = os.path.join(project_dir, "Results")
EXPERIMENT_OUTPUT_DIR = os.path.join(project_dir, "Final_Results")
MODEL_NAME = "resnet18.a1_in1k"

In [ ]:
# Mount google drive to local dir
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Dependencies

In [ ]:
import argparse

import torch
import torch.optim as optim
from torch import nn
from torch.nn import Module
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision import models
from tqdm import tqdm
from tqdm.auto import tqdm
import timm
import wandb
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from einops import rearrange
from scipy.optimize import minimize
import pandas as pd


#0. Spawrious Source Code

In [ ]:
import os
import tarfile
import urllib
import urllib.request
from typing import Any, Tuple

import torch
from PIL import Image
from torch.utils.data import ConcatDataset, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import timm
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# MODEL_NAME = "vit_so400m_patch14_siglip_384"
# MODEL_NAME = 'swin_base_patch4_window7_224.ms_in22k_ft_in1k'
# MODEL_NAME = 'deit3_base_patch16_224.fb_in22k_ft_in1k'
# MODEL_NAME = 'beit_base_patch16_224.in22k_ft_in22k_in1k'
# MODEL_NAME = 'eva02_base_patch14_448.mim_in22k_ft_in22k_in1k'
# MODEL_NAME = 'levit_128s.fb_dist_in1k'

def set_model_name(name):
    global MODEL_NAME
    MODEL_NAME = name


def _extract_dataset_from_tar(
    tar_file_name: str, data_dir: str
) -> None:
    tar_file_dst = os.path.join(data_dir, tar_file_name)
    print("Extracting dataset...")
    tar = tarfile.open(tar_file_dst, "r:gz")
    tar.extractall(os.path.dirname(tar_file_dst))
    tar.close()


def _download_dataset_if_not_available(
    dataset_name: str, data_dir: str, remove_tar_after_extracting: bool = True
) -> None:
    """
    datasets.txt file, which is present in the data_dir, is used to check if the dataset is already extracted. If the dataset is already extracted, then the tar file is not downloaded again.
    """
    data_dir = data_dir.split("/spawrious224/")[
        0
    ]  # in case people pass in the wrong root_dir
    os.makedirs(data_dir, exist_ok=True)
    dataset_name = dataset_name.lower()
    if dataset_name.split("_")[0] == "m2m":
        dataset_name = "entire_dataset"
    url_dict = {
        "entire_dataset": "https://www.dropbox.com/s/hofkueo8qvaqlp3/spawrious224__entire_dataset.tar.gz?dl=1",
        "o2o_easy": "https://www.dropbox.com/s/kwhiv60ihxe3owy/spawrious224__o2o_easy.tar.gz?dl=1",
        "o2o_medium": "https://www.dropbox.com/s/x03gkhdwar5kht4/spawrious224__o2o_medium.tar.gz?dl=1",
        "o2o_hard": "https://www.dropbox.com/s/p1ry121m2gjj158/spawrious224__o2o_hard.tar.gz?dl=1",
        # "m2m": "https://www.dropbox.com/s/5usem63nfub266y/spawrious__m2m.tar.gz?dl=1",
    }
    tar_file_name = f"spawrious224__{dataset_name}.tar.gz"
    tar_file_dst = os.path.join(data_dir, tar_file_name)
    url = url_dict[dataset_name]

    # check if the dataset is already extracted
    if _check_images_availability(data_dir, dataset_name):
        print("Dataset already downloaded and extracted.")
        return
    # check if the tar file is already downloaded
    else:
        if os.path.exists(tar_file_dst):
            print("Dataset already downloaded. Extracting...")
            _extract_dataset_from_tar(
                tar_file_name, data_dir
            )
            return
        # download the tar file and extract from it
        else:
            print("Dataset not found. Downloading...")
            response = urllib.request.urlopen(url)
            total_size = int(response.headers.get("Content-Length", 0))
            block_size = 1024
            # Track progress of download
            progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True)
            with open(tar_file_dst, "wb") as f:
                while True:
                    buffer = response.read(block_size)
                    if not buffer:
                        break
                    f.write(buffer)
                    progress_bar.update(len(buffer))
            progress_bar.close()
            print("Dataset downloaded. Extracting...")
            _extract_dataset_from_tar(
                tar_file_name, data_dir
            )
            return


class CustomImageFolder(Dataset):
    """
    A class that takes one folder at a time and loads a set number of images in a folder and assigns them a specific class
    """

    def __init__(
        self, folder_path, class_index, location_index, limit=None, transform=None
    ):
        self.folder_path = folder_path
        self.class_index = class_index
        self.location_index = location_index
        self.image_paths = [
            os.path.join(folder_path, img)
            for img in os.listdir(folder_path)
            if img.endswith((".png", ".jpg", ".jpeg"))
        ]
        if limit:
            self.image_paths = self.image_paths[:limit]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Tuple[Any, Any, Any]:
        img_path = self.image_paths[index]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        class_label = torch.tensor(self.class_index, dtype=torch.long)
        location_label = torch.tensor(self.location_index, dtype=torch.long)
        return img, class_label, location_label


class MultipleDomainDataset:
    N_STEPS = 5001  # Default, subclasses may override
    CHECKPOINT_FREQ = 100  # Default, subclasses may override
    N_WORKERS = 8  # Default, subclasses may override
    ENVIRONMENTS = None  # Subclasses should override
    INPUT_SHAPE = None  # Subclasses should override

    def __getitem__(self, index):
        return self.datasets[index]

    def __len__(self):
        return len(self.datasets)


def build_combination(benchmark_type, group, test, filler=None):
    total = 3168
    combinations = {}
    if "m2m" in benchmark_type:
        counts = [total, total]
        combinations["train_combinations"] = {
            ("bulldog",): [(group[0], counts[0]), (group[1], counts[1])],
            ("dachshund",): [(group[1], counts[0]), (group[0], counts[1])],
            ("labrador",): [(group[2], counts[0]), (group[3], counts[1])],
            ("corgi",): [(group[3], counts[0]), (group[2], counts[1])],
        }
        combinations["test_combinations"] = {
            ("bulldog",): [test[0], test[1]],
            ("dachshund",): [test[1], test[0]],
            ("labrador",): [test[2], test[3]],
            ("corgi",): [test[3], test[2]],
        }
    else:
        counts = [int(0.97 * total), int(0.87 * total)]
        combinations["train_combinations"] = {
            ("bulldog",): [(group[0], counts[0]), (group[0], counts[1])],
            ("dachshund",): [(group[1], counts[0]), (group[1], counts[1])],
            ("labrador",): [(group[2], counts[0]), (group[2], counts[1])],
            ("corgi",): [(group[3], counts[0]), (group[3], counts[1])],
            ("bulldog", "dachshund", "labrador", "corgi"): [
                (filler, total - counts[0]),
                (filler, total - counts[1]),
            ],
        }
        combinations["test_combinations"] = {
            ("bulldog",): [test[0], test[0]],
            ("dachshund",): [test[1], test[1]],
            ("labrador",): [test[2], test[2]],
            ("corgi",): [test[3], test[3]],
        }
    return combinations


def _get_combinations(benchmark_type: str) -> Tuple[dict, dict]:
    combinations = {
        "o2o_easy": (
            ["desert", "jungle", "dirt", "snow"],
            ["dirt", "snow", "desert", "jungle"],
            "beach",
        ),
        "o2o_medium": (
            ["mountain", "beach", "dirt", "jungle"],
            ["jungle", "dirt", "beach", "snow"],
            "desert",
        ),
        "o2o_hard": (
            ["jungle", "mountain", "snow", "desert"],
            ["mountain", "snow", "desert", "jungle"],
            "beach",
        ),
        "m2m_hard": (
            ["dirt", "jungle", "snow", "beach"],
            ["snow", "beach", "dirt", "jungle"],
            None,
        ),
        "m2m_easy": (
            ["desert", "mountain", "dirt", "jungle"],
            ["dirt", "jungle", "mountain", "desert"],
            None,
        ),
        "m2m_medium": (
            ["beach", "snow", "mountain", "desert"],
            ["desert", "mountain", "beach", "snow"],
            None,
        ),
    }
    if benchmark_type not in combinations:
        raise ValueError("Invalid benchmark type")
    group, test, filler = combinations[benchmark_type]
    return build_combination(benchmark_type, group, test, filler)


class SpawriousBenchmark(MultipleDomainDataset):
    ENVIRONMENTS = ["Test", "SC_group_1", "SC_group_2"]
    input_shape = (3, 224, 224)
    num_classes = 4
    class_list = ["bulldog", "corgi", "dachshund", "labrador"]
    locations_list = ["desert", "jungle", "dirt", "mountain", "snow", "beach"]

    def __init__(self, benchmark, root_dir, augment=True):
        combinations = _get_combinations(benchmark.lower())
        self.type1 = benchmark.lower().startswith("o2o")
        train_datasets, test_datasets = self._prepare_data_lists(
            combinations["train_combinations"],
            combinations["test_combinations"],
            root_dir,
            augment,
        )
        self.datasets = [ConcatDataset(test_datasets)] + train_datasets

    def get_train_dataset(self):
        return torch.utils.data.ConcatDataset(self.datasets[1:])

    def get_test_dataset(self):
        return self.datasets[0]

    # Prepares the train and test data lists by applying the necessary transformations.
    def _prepare_data_lists(
        self, train_combinations, test_combinations, root_dir, augment
    ):
        backbone = timm.create_model(
            # "vit_so400m_patch14_siglip_384",
            MODEL_NAME,
            pretrained=True,
            num_classes=0,
        ).eval()
        self.data_config = timm.data.resolve_model_data_config(backbone)
        test_transforms = timm.data.create_transform(
            **self.data_config, is_training=False
        )

        # test_transforms = transforms.Compose(
        #     [
        #         transforms.Resize((self.input_shape[1], self.input_shape[2])),
        #         transforms.transforms.ToTensor(),
        #         transforms.Normalize(
        #             mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        #         ),
        #     ]
        # )

        if augment:
            train_transforms = timm.data.create_transform(
                **self.data_config, is_training=True
            )
        else:
            train_transforms = test_transforms
        print("Creating Training Dataset:")
        train_data_list = self._create_data_list(
            train_combinations, root_dir, train_transforms
        )
        print("Creating Testing Dataset:")
        test_data_list = self._create_data_list(
            test_combinations, root_dir, test_transforms
        )

        return train_data_list, test_data_list

    # Creates a list of datasets based on the given combinations and transformations.
    def _create_data_list(self, combinations, root_dir, transforms):
        data_list = []
        if isinstance(combinations, dict):
            # Build class groups for a given set of combinations, root directory, and transformations.
            for_each_class_group = []
            cg_index = 0
            for classes, comb_list in combinations.items():
                for_each_class_group.append([])
                for ind, location_limit in enumerate(comb_list):
                    if isinstance(location_limit, tuple):
                        location, limit = location_limit
                    else:
                        location, limit = location_limit, None
                    cg_data_list = []
                    for cls in classes:
                        path = os.path.join(
                            root_dir,
                            "spawrious224",
                            f"{0 if not self.type1 else ind}/{location}/{cls}",
                        )
                        print(f"    Combination: {0 if not self.type1 else ind}/{location}/{cls}")
                        print(f"    Limit: {limit}")
                        data = CustomImageFolder(
                            folder_path=path,
                            class_index=self.class_list.index(cls),
                            location_index=self.locations_list.index(location),
                            limit=limit,
                            transform=transforms,
                        )
                        cg_data_list.append(data)

                    for_each_class_group[cg_index].append(ConcatDataset(cg_data_list))
                cg_index += 1

            for group in range(len(for_each_class_group[0])):
                data_list.append(
                    ConcatDataset(
                        [
                            for_each_class_group[k][group]
                            for k in range(len(for_each_class_group))
                        ]
                    )
                )
        else:
            for location in combinations:
                path = os.path.join(root_dir, f"{0}/{location}/")
                data = ImageFolder(root=path, transform=transforms)
                data_list.append(data)

        return data_list


def _check_images_availability(root_dir: str, dataset_type: str) -> bool:
    # Get the combinations for the given dataset type
    root_dir = root_dir.split("/spawrious224/")[
        0
    ]  # in case people pass in the wrong root_dir
    if dataset_type == "entire_dataset":
        for dataset in ["0", "1", "domain_adaptation_ds"]:
            for location in ["snow", "jungle", "desert", "dirt", "mountain", "beach"]:
                for cls in ["bulldog", "corgi", "dachshund", "labrador"]:
                    path = os.path.join(
                        root_dir, "spawrious224", f"{dataset}/{location}/{cls}"
                    )
                    if not os.path.exists(path) or not any(
                        img.endswith((".png", ".jpg", ".jpeg"))
                        for img in os.listdir(path)
                    ):
                        return False
        return True
    combinations = _get_combinations(dataset_type.lower())

    # Extract the train and test combinations
    train_combinations = combinations["train_combinations"]
    test_combinations = combinations["test_combinations"]

    # Check if the relevant images for each combination are present in the root directory
    for combination in [train_combinations, test_combinations]:
        for classes, comb_list in combination.items():
            for ind, location_limit in enumerate(comb_list):
                if isinstance(location_limit, tuple):
                    location, limit = location_limit
                else:
                    location, limit = location_limit, None

                for cls in classes:
                    path = os.path.join(
                        root_dir,
                        "spawrious224",
                        f"{0 if not dataset_type.lower().startswith('o2o') else ind}/{location}/{cls}",
                    )

                    # If the path does not exist or there are no relevant images, return False
                    if not os.path.exists(path) or not any(
                        img.endswith((".png", ".jpg", ".jpeg"))
                        for img in os.listdir(path)
                    ):
                        return False

    # If all the required images are present, return True
    return True


def get_spawrious_dataset(root_dir: str, dataset_name: str = "entire_dataset"):
    """
    Returns the dataset as a torch dataset, and downloads dataset if dataset is not already available.

    By default, the entire dataset is downloaded, which is necessary for m2m experiments, and domain adaptation experiments
    """
    root_dir = root_dir.split("/spawrious224/")[
        0
    ]  # in case people pass in the wrong root_dir
    assert dataset_name.lower() in {
        "o2o_easy",
        "o2o_medium",
        "o2o_hard",
        "m2m_easy",
        "m2m_medium",
        "m2m_hard",
        "m2m",
        "entire_dataset",
    }, f"Invalid dataset type: {dataset_name}"
    _download_dataset_if_not_available(dataset_name, root_dir)
    # TODO: get m2m to use entire dataset, not half of it
    return SpawriousBenchmark(dataset_name, root_dir, augment=True)

#1. Load Dataset

In [ ]:
# Mount google drive to local dir
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Create data dir
!mkdir /content/data

In [ ]:
# Set Model Name (using global var)
set_model_name(MODEL_NAME)

import tarfile

def download_dataset(dataset_name):
  # Unzip the  dataset
  drive_dir = os.path.join(datasets_dir, dataset_name)
  !cp $drive_dir /content/data
  tar_file = os.path.join('/content/data', drive_dir.split('/')[-1])
  data_dir = "/content/data/"

  # Extract the dataset
  with tarfile.open(tar_file, 'r:gz') as tar:  # Use 'r' for .tar and 'r:gz' for .tar.gz
      tar.extractall(path=data_dir)

# Download the datasets
download_dataset("spawrious224__o2o_easy.tar.gz")
download_dataset("background_only_dataset.tar.gz")

# 2. Download both Models
models are saved to variables 'normal_model' and 'bugged_model'

In [ ]:
def load_model(model_name, model_type="resnet18.a1_in1k"):

    checkpoint_path = os.path.join(models_dir, model_name)

    if model_type != "resnet18.a1_in1k":
        print("Only resnet18 is supported currently")
        return

    # Checks if GPU is available first
    if torch.cuda.is_available():
        model_weights = torch.load(os.path.join(models_dir, checkpoint_path))
    else:
        model_weights = torch.load(os.path.join(models_dir, checkpoint_path), map_location=torch.device('cpu'))

    # Create model backbone (base model)
    model = timm.create_model(
                model_type,
                pretrained=True,
                num_classes=4).eval()

    # Load weights
    model.load_state_dict(model_weights)

    # set ENTIRE model to eval mode
    model.eval()

    # Switch weights to cuda if possible
    if torch.cuda.is_available():
        model.to('cuda')

    return model


In [ ]:
normal_model = load_model(normal_model_name, "resnet18.a1_in1k")
bugged_model = load_model(bugged_model_name, "resnet18.a1_in1k")

<ipython-input-9-a9afaae834bd>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load(os.path.join(models_dir, checkpoint_path))
/usr/local/lib/python3

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

#3. Concept Extraction with Nonnegative Matrix Factorization

## Helpers

In [ ]:
import cv2
from skimage import measure
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np


def set_size(w,h):
  """Set matplot figure size"""
  plt.rcParams["figure.figsize"] = [w,h]

def show(img, norm=True, **kwargs):
  """ Display torch/tf tensor """
  if isinstance(img, torch.Tensor):
    img = img.detach().cpu().numpy()
  img = np.array(img, dtype=np.float32)

  # check if channel first
  if img.shape[0] == 1:
    img = img[0]
  if img.shape[0] == 3:
    img = np.moveaxis(img, 0, 2)
  # check if cmap
  if img.shape[-1] == 1:
    img = img[:,:,0]
  # normalize
  if norm:
    img -= img.min(); img/=img.max()

  plt.imshow(img, **kwargs)
  plt.axis('off')
  plt.grid(None)


all_cmaps = []

def get_alpha_cmap(rgb_color):
    # Normalize the RGB color values to the range [0, 1]
    rgb_color = np.array(rgb_color) / 255.0

    # Create an alpha colormap with varying alpha values
    cmap_data = {'red':   [(0, rgb_color[0], rgb_color[0]),
                           (1, rgb_color[0], rgb_color[0])],
                 'green': [(0, rgb_color[1], rgb_color[1]),
                           (1, rgb_color[1], rgb_color[1])],
                 'blue':  [(0, rgb_color[2], rgb_color[2]),
                           (1, rgb_color[2], rgb_color[2])],
                 'alpha': [(0, 0, 0),
                           (1, 1, 1)]}

    alpha_cmap = mcolors.LinearSegmentedColormap('alpha_cmap', cmap_data, N=256)

    all_cmaps.append(alpha_cmap)

    return alpha_cmap

slack_blue  = get_alpha_cmap((54, 197, 240))
slack_green = get_alpha_cmap((46, 182, 125) )
slack_red   = get_alpha_cmap((210, 40, 95))
slack_yellow = get_alpha_cmap((236, 178, 46))
slack_violet = get_alpha_cmap((84, 25, 85))

google_blue = get_alpha_cmap((66, 133, 244))
google_red = get_alpha_cmap((219, 68, 55))
google_yellow= get_alpha_cmap((244, 180, 0))
google_green = get_alpha_cmap((15, 157, 88))

deep_purple = get_alpha_cmap((103, 58, 183))
pink = get_alpha_cmap((236, 64, 122))
anthracite = get_alpha_cmap((13, 13, 21))

palette_1 = get_alpha_cmap((33, 115, 174))
palette_2 = get_alpha_cmap((216, 146, 49))
palette_3 = get_alpha_cmap((58, 155, 118))
palette_4 = get_alpha_cmap((210, 158, 201))

cmaps = [google_blue, slack_green, slack_red, slack_yellow, slack_violet, anthracite, google_blue, pink, deep_purple]


def create_alpha_cmap(color_input, name=None):
    if isinstance(color_input, str):
        # If the input is a colormap name
        base_cmap = plt.cm.get_cmap(color_input)
    elif isinstance(color_input, tuple) and len(color_input) == 3:
        if np.max(color_input) > 1:
            color_input = (
                color_input[0] / 255,
                color_input[1] / 255,
                color_input[2] / 255,
            )
        if name is None:
            name = f'RGB{color_input}'
        base_cmap = mcolors.LinearSegmentedColormap.from_list(name, [color_input, color_input])
    else:
        raise ValueError("Invalid color_input. Must be a colormap name or an RGB tuple.")

    # alpha values ranging from 0 to 1
    # tfel: useful to start above zero?
    alpha = np.linspace(0.0, 1, base_cmap.N)

    colors = base_cmap(np.arange(base_cmap.N))
    colors[:, -1] = alpha

    alpha_cmap = mcolors.LinearSegmentedColormap.from_list('alpha_cmap', colors)

    return alpha_cmap


# predefine some alpha cmaps
JET_ALPHA = create_alpha_cmap('jet')
VIRIDIS_ALPHA = create_alpha_cmap('viridis')

_tab_10_colors = [
    (31, 119, 180),   # tab:blue
    (255, 127, 14),   # tab:orange
    (44, 160, 44),    # tab:green
    (214, 39, 40),    # tab:red
    (148, 103, 189),  # tab:purple
    (140, 86, 75),    # tab:brown
    (227, 119, 194),  # tab:pink
    (127, 127, 127),  # tab:gray
    (188, 189, 34),   # tab:olive
    (23, 190, 207)    # tab:cyan
]

TAB10_ALPHA = [create_alpha_cmap(color) for color in _tab_10_colors]

cmaps = [google_blue, slack_green, slack_red, slack_yellow, slack_violet, anthracite, google_blue, pink, deep_purple]
cmaps = cmaps + TAB10_ALPHA


def _get_representative_ids(heatmaps, concept_id, num_images=8):
    return np.mean(heatmaps[:, :, :, concept_id], axis=(1, 2)).argsort()[-num_images:]


# Overlays heatmaps of the concept in the image
def overlay_top_heatmaps(images, heatmaps, concept_id, cmap=None, alpha=0.35, num_images=8):
    assert len(images) == len(heatmaps)
    assert heatmaps.shape[-1] > concept_id
    assert heatmaps.ndim == 4

    if cmap is None:
        cmap = TAB10_ALPHA[concept_id] if heatmaps.shape[-1] < 10 else VIRIDIS_ALPHA
        alpha = 1.0

    best_ids = _get_representative_ids(heatmaps, concept_id, num_images)

    for i, idx in enumerate(best_ids):
        image = images[idx]
        heatmap = cv2.resize(heatmaps[idx, :, :, concept_id], (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)
        #raise NotImplementedError("re implement cv2 resize with bicubic")

        plt.subplot(2, int(num_images/2), i + 1)
        plt.tight_layout()
        show(image)
        show(heatmap, cmap=cmap, alpha=alpha)

def evidence_top_images(images, heatmaps, concept_id, num_images=8, percentiles=None):
    assert len(images) == len(heatmaps)
    assert heatmaps.shape[-1] > concept_id
    assert heatmaps.ndim == 4

    if percentiles is None:
        percentiles = np.linspace(50, 95, 10)

    best_ids = _get_representative_ids(heatmaps, concept_id, num_images)

    for i, idx in enumerate(best_ids):
        image = images[idx]
        heatmap = cv2.resize(heatmaps[idx, :, :, concept_id], (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)

        mask = np.zeros_like(heatmap)
        for percentile in percentiles:
            mask[heatmap > np.percentile(heatmap, percentile)] += 1.0
        mask = mask / len(percentiles)

        masked_image = image * mask[:, :, None]

        plt.subplot(2, int(num_images/2), i + 1)
        plt.tight_layout()
        show(masked_image.astype(np.uint8))

def zoom_top_images(images, heatmaps, concept_id, zoom_size=100, num_images=8):
    assert len(images) == len(heatmaps)
    assert heatmaps.shape[-1] > concept_id
    assert heatmaps.ndim == 4

    best_ids = _get_representative_ids(heatmaps, concept_id, num_images)

    for i, idx in enumerate(best_ids):
        image = images[idx]
        heatmap = cv2.resize(heatmaps[idx, :, :, concept_id], (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)

        hottest_point = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)

        x_min = max(hottest_point[0] - zoom_size // 2, 0)
        x_max = min(hottest_point[0] + zoom_size // 2, image.shape[0])
        y_min = max(hottest_point[1] - zoom_size // 2, 0)
        y_max = min(hottest_point[1] + zoom_size // 2, image.shape[1])

        zoomed_image = image[x_min:x_max, y_min:y_max]

        plt.subplot(2, int(num_images/2), i + 1)
        show(zoomed_image)

# Produces outlines of each concept
def contour_top_image(images, heatmaps, concept_id, percentiles=None, cmap="viridis", linewidth=1.0, num_images=8):
    assert len(images) == len(heatmaps)
    assert heatmaps.shape[-1] > concept_id
    assert heatmaps.ndim == 4

    if percentiles is None:
        percentiles = [70]

    best_ids = _get_representative_ids(heatmaps, concept_id, num_images)

    for i, idx in enumerate(best_ids):
        image = images[idx]
        plt.subplot(2, int(num_images/2), i + 1)
        show(image)

        heatmap = cv2.resize(heatmaps[idx, :, :, concept_id], (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)

        for percentile in percentiles:
            cut_value = np.percentile(heatmap, percentile)
            contours = measure.find_contours(heatmap, cut_value)
            for contour in contours:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=linewidth, color=plt.get_cmap(cmap)(percentile / 100))




<ipython-input-38-a9b8f2bf6f2f>:83: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  base_cmap = plt.cm.get_cmap(color_input)


In [ ]:
def save_craft_results(model_checkpoint, class_idx, concepts, importance, heatmaps):
  # Create Directory, change name as needed
  dir_name = f"CRAFT_{class_idx}_class_{heatmaps.shape[0]}_im_{concepts.shape[0]}_concepts_{model_checkpoint}"
  results_dir = os.path.join(project_dir, "Results", dir_name)

  os.makedirs(results_dir, exist_ok=True)

  # Save concepts
  np.save(os.path.join(results_dir, "Concepts.npy"), concepts)
  # Save importances
  np.save(os.path.join(results_dir, "Importances.npy"), importance)
  # Heatmaps (activations)
  np.save(os.path.join(results_dir, "Activations.npy"), heatmaps)

In [ ]:
import torch
from sklearn.decomposition import NMF
import torchvision
import timm
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import ceil
import numpy as np

def get_images(data_loader, num_images):
    X = []
    n = 0
    for images, classes, locations in data_loader:
        X.append(images)
        n += data_loader.batch_size
        if n >= num_images:
            return X

def get_stacked_activations(model, X, num_images):
    # Run forward pass
    A = []
    for batch in X:
        # batch = transform(batch)
        batch = batch.to('cuda')
        batch_A = model.forward_features(batch)
        A.append(batch_A.detach().cpu().numpy())

    A = np.concatenate(A, axis=0)
    return A

def load_images_as_np(data_list, num_images=300):
  X = []
  n = 0
  for entry in data_list:
    X.append(entry[0])
    n += 1
    if n == num_images:
      return np.array(X)


def get_transforms(model):
  # Gather transforms
  config = timm.data.resolve_model_data_config(model)
  transform = timm.data.create_transform(**config, is_training=False,)
  return transform

##Implementation

In [67]:
def get_craft_concepts(model, model_checkpoint, class_dir, class_idx, num_concepts=8, num_images=300, results_dir=RESULTS_DIR, visualize=False):
  transform = get_transforms(model)


  # Create a custom image folder (the Spawrious custom class for datasets)
  class_data_list = CustomImageFolder(class_dir,
                                    class_index=1, location_index=4,
                                    transform=transform, limit =num_images)
  no_transforms_list = CustomImageFolder(class_dir,
                                    class_index=1, location_index=4,
                                    transform=None, limit=num_images)

  # Create a dataloader
  class_loader = torch.utils.data.DataLoader(
      class_data_list,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  # get images
  X = get_images(class_loader, num_images)
  A = get_stacked_activations(model, X, num_images)

  # Convert X to tensor
  X_cat = torch.cat(X, dim=0)
  nmf = NMF(num_concepts)

  A_flatten = rearrange(A, 'n c h w -> (n h w) c')

  Z = nmf.fit_transform(A_flatten)
  D = nmf.components_

  Z_heatmaps = rearrange(Z, '(n h w) r -> n h w r', n=A.shape[0], h=7, w=7)
  # most important are E(Z) * W'
  # W' is D @ W.T
  W = model.fc.weight.detach().cpu().numpy()
  W_p = D @ W.T
  Importance = np.mean(Z, 0) * W_p[:, class_idx]

  # Plot concepts
  Imgs = load_images_as_np(no_transforms_list, num_images)

  for concept_id in np.argsort(Importance)[::-1]:

    set_size(8, 4)

    overlay_top_heatmaps(Imgs[:A.shape[0]], Z_heatmaps, concept_id, cmap=cmaps[concept_id], alpha=1.0)
    plt.suptitle(f"Concept {concept_id}; Importance:{Importance[concept_id]:.4f}")
    plt.tight_layout()
    plt.axis('off')

    # always save the concepts as pngs: save to a specifically-name dir
    dir_name = f"CRAFT_{class_idx}_class_{num_images}_im_{D.shape[0]}_concepts_{model_checkpoint}"
    plot_name = f"concept_{concept_id}_importance_{Importance[concept_id]:.4f}.png"
    plot_path = os.path.join(results_dir, dir_name, "concept_images", plot_name)
    os.makedirs(os.path.join(results_dir, dir_name), exist_ok=True)
    os.makedirs(os.path.join(results_dir, dir_name, "concept_images"), exist_ok=True)


    plt.savefig(plot_path, dpi=300)

    if visualize:
      plt.show()
      print("\n")


  # Save results
  save_craft_results(model_checkpoint, class_idx, D, Importance, Z_heatmaps)
  return dir_name


In [68]:
get_craft_concepts(normal_model, normal_model_name, "/content/data/spawrious224/0/desert/bulldog", 0, num_concepts=8, num_images=10, results_dir=RESULTS_DIR, visualize=True)

Output hidden; open in https://colab.research.google.com to view.

# 4. Load Concepts

In [ ]:
# Custom class for storing concept results
class ConceptResults():

    def __init__(self, concept_instance_name):
        concept_results_dir = os.path.join(RESULTS_DIR, concept_instance_name)
        self.concepts = np.load(os.path.join(concept_results_dir, "Concepts.npy"))
        self.importances = np.load(os.path.join(concept_results_dir, "Importances.npy")),
        self.activations = np.load(os.path.join(concept_results_dir, "Activations.npy")),



def load_concept_results(concept_instance_name):
    """Load pre-computed concept results from the results directory"""
    concept_results_dir = os.path.join(RESULTS_DIR, concept_instance_name)

    results = {
        'concepts': np.load(os.path.join(concept_results_dir, "Concepts.npy")),
        'importances': np.load(os.path.join(concept_results_dir, "Importances.npy")),
        'activations': np.load(os.path.join(concept_results_dir, "Activations.npy")),
    }

    return results


# 5. Run Experiments

## Helper Functions

In [ ]:
# Convert images from a dataloader into a list of tensors
def get_images(data_loader, num_images):
    X = []
    n = 0
    for images, classes, locations in data_loader:
        X.append(images)
        n += data_loader.batch_size
        if n >= num_images:
            return X

# Store the stacked activations of
def get_stacked_activations(model, X, num_images):
    # Run forward pass
    A = []
    for batch in X:
        batch = get_transforms(model)(batch)
        batch = batch.to('cuda')
        batch_A = model.forward_features(batch)
        A.append(batch_A.detach().cpu().numpy())

    A = np.concatenate(A, axis=0)
    return A

# Helper : computes the cost of an intermediary nmf solution
def nmf_cost(H_vec, W, A_T):
    k, n = A_T.shape[0], W.shape[0]
    H = H_vec.reshape(k, n)

    reconstruction = H @ W
    return np.linalg.norm(A_T - reconstruction, 'fro')**2


# Get Activations of a set of concepts when passing through a new set of images
def get_concept_activations(stacked_activations, concepts):

    # Flatten the activations
    A_flatten = rearrange(stacked_activations, 'n c h w -> (n h w) c')

    # Initialize random guess for concept activations
    concept_activation_init = np.random.rand(A_flatten.shape[0], concepts.shape[0])
    print("H init", concept_activation_init.shape)
    concept_activation_init = concept_activation_init.flatten()

    # Solve with constraint H' >= 0
    result = minimize(nmf_cost, concept_activation_init,
                      args=(concepts, A_flatten),
                      method='L-BFGS-B',
                      bounds=[(0, None)] * concept_activation_init.size,
                      options={'maxiter':200,'disp': True,})
    concept_activations = result.x.reshape(concepts.shape[1], -1)

    return concept_activations

def get_concept_activations_gpu(stacked_activations, concepts):

    # Flatten the activations
    A_flatten = rearrange(stacked_activations, 'n c h w -> (n h w) c')

    # Move matrices to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    A_torch = torch.tensor(A_flatten, dtype=torch.float32).to(device)  # Transposed A
    W_torch = torch.tensor(concepts, dtype=torch.float32).to(device)
    # H_prime: randomly-initialized matrix for the concept activations
    H_prime = torch.nn.Parameter(torch.rand(A_torch.shape[0], W_torch.shape[0], device=device, dtype=torch.float32))

    # Checkout shapes of matrices
    # print(f"Activations: {A_torch.shape}, concepts:  {W_torch.shape}, concept activations: {H_prime.shape}")

    # Define loss function (Frobenius norm)
    loss_fn = nn.MSELoss()

    optimizer = torch.optim.LBFGS([H_prime], max_iter=300)

    def closure():
        optimizer.zero_grad()
        reconstruction =  H_prime @ W_torch
        loss = loss_fn(reconstruction, A_torch)
        loss.backward()
        return loss

    # Run optimization
    optimizer.step(closure)

    # Convert back to NumPy if needed
    H_prime_result = H_prime.detach().cpu().numpy()
    return H_prime_result


# Return concept importances
def get_concept_importances(model, concepts, concept_activations, class_idx):
    # Pull final layer weights from model
    W = model.fc.weight.detach().cpu().numpy()

    # Multiply those by the concepts -- this provides the importance of each concept
    W_p = concepts @ W.T

    # Average the weights * activations specifically for the target class
    importances = np.mean(concept_activations, 0) * W_p[:, class_idx] # corgi class

    return importances

## First, check that models are spurious

In [183]:
# Eval loop
def evaluate(model: Module, loader: DataLoader, device: torch.device) -> float:
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels, _ in tqdm(
            loader, desc="Evaluating", leave=False
        ):  # third item is the location label
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            #print("\nPredictions: ",  predicted, "\nTruth: ", labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc

# Return the class index associated with a specific location
def get_class_index(model_name, location):
  # Extract the trainig dataset type from the model_name
  if "o2o" in model_name:
    train_data_type = "o2o_"+model_name.split("o2o_")[1].split("_")[0]
  else:
    train_data_type = "entire_dataset"

  # Mapping of (idx in the combinations below) : (idx of class)
  # example: corgi uses the "snow" class in o2o_easy (3rd index), but is has class_idx 1
  class_index_mappings = {0: 0, 1: 2, 2: 3, 3: 1}

  combinations = {
        "o2o_easy": (
            ["desert", "jungle", "dirt", "snow"],
        ),

        "entire_dataset": (
            ["desert", "jungle", "dirt", "snow"],
        ),

        "o2o_medium": (
            ["mountain", "beach", "dirt", "jungle"],
        ),
        "o2o_hard": (
            ["jungle", "mountain", "snow", "desert"],
        ),
    }

  try:
    return class_index_mappings[combinations[train_data_type][0].index(location)]
  except:
    return -1

# Test the model on a background-only dataset
def bg_only_test(model, model_name, bg_dir, num_images=100):

  data_lists = []
  # Create a custom image folder (the Spawrious custom class for datasets)
  for location_dir in os.listdir(bg_dir):
      location = location_dir.split('_')[0]
      location_dir = os.path.join(bg_dir, location_dir)

      # Index into combinations to get the class index associated with the background
      class_index = get_class_index(model_name, location)
      if class_index == -1:
        continue

      background_data_list = CustomImageFolder(location_dir,
                                        class_index=class_index, location_index=0,
                                        transform = get_transforms(model), limit = num_images)
      # Add it to the list of datasets
      data_lists.append(background_data_list)


  # Concatenate all of the datasets
  combined_dataset = torch.utils.data.ConcatDataset(data_lists)

  # Create a dataloader
  bg_loader = torch.utils.data.DataLoader(
      combined_dataset,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  return evaluate(model=model, loader=bg_loader, device=torch.device("cuda"))

## Method 1: Testing Background Concept Importance

In [102]:
# Filter out background concepts, using activations on the class dataset and background-only dataset
def filter_background_concepts(activations, background_activations, concepts, num_images, cutoff = 0.5, method='mean', verbose=False):

    # Iterate through concepts
    background_concepts = []

    # Dict for saving results
    results = {"concept" : [],
              "class_activation" : [],
              "background_activation" : [],
              "ratio" : [],
               "background" : []
             }

    for concept_id in range(len(concepts)):
        if method == 'mean':

            # Compute mean activation of that concept
            activation_aggregated = activations[..., concept_id].mean()
            bg_activation_aggregated = background_activations[..., concept_id].mean()

        if method == 'median':

            # Compute median activation of that concept
            activation_aggregated = np.median(activations[..., concept_id])
            bg_activation_aggregated = np.median(background_activations[..., concept_id])

        if method == 'med-mean':

            # Take mean of each 7*7 activation map, then median of those means
            activation_aggregated = np.median(np.mean(activations[..., concept_id], axis=(1,2)))
            bg_activation_aggregated = np.median(np.mean(background_activations[..., concept_id], axis=(1,2)))


        # If the concept is more activated by the background, then it
        # likely does not have anything to do with the class
        ratio = activation_aggregated / bg_activation_aggregated
        if ratio < cutoff:
            background_concepts.append(concept_id)

        # Update the results dict
        results["concept"].append(concept_id)
        results["class_activation"].append(activation_aggregated)
        results["background_activation"].append(bg_activation_aggregated)
        results["ratio"].append(ratio)
        results["background"].append(ratio < cutoff)




    # Return concepts
    return background_concepts, pd.DataFrame(results)

def score(importances, bg_concepts):
  background_concept_importances = np.linalg.norm(importances[bg_concepts])
  return background_concept_importances / np.linalg.norm(importances)

In [ ]:
def show_concepts(heatmaps, importances, imgs):
  for concept_id in np.argsort(importances)[::-1]:
    set_size(10, 3)
    overlay_top_heatmaps(imgs[:heatmaps.shape[0]], heatmaps, concept_id, cmap=cmaps[concept_id], alpha=1.0)
    print(f'Concept {concept_id} Importance score', importances[concept_id])
    plt.tight_layout()
    plt.show()


def compare_concept(concept_id, heatmaps, bg_heatmaps, imgs, bg_imgs, importances):
    set_size(10, 3)
    overlay_top_heatmaps(imgs[:heatmaps.shape[0]], heatmaps, concept_id, alpha=1.0, num_images=6)
    #zoom_top_images(imgs[:heatmaps.shape[0]], heatmaps, concept_id)

    print(f'Class Data: Concept {concept_id} Importance score', importances[concept_id])
    plt.tight_layout()
    plt.show()
    overlay_top_heatmaps(bg_imgs[:heatmaps.shape[0]], bg_heatmaps, concept_id, alpha=1.0, num_images=6)
    #zoom_top_images(bg_imgs[:heatmaps.shape[0]], bg_heatmaps, concept_id)

    print(f'Background Data: Concept {concept_id} Importance score', importances[concept_id])
    plt.tight_layout()
    plt.show()

In [111]:
def experiment(bugged_model, normal_model,
               class_data_dir, bg_data_dir,
               bugged_concept_results, normal_concept_results, class_id,
               num_images = 100, show_images = True, method='med-mean', cutoff = 1.0):

  # Create a custom image folder (the Spawrious custom class for datasets)
  class_data_list = CustomImageFolder(class_data_dir,
                                    class_index=1, location_index=4,
                                    transform = get_transforms(bugged_model), limit = num_images)
  # Create custom image folder for snow-only data ; these are also labeled as corgi
  background_data_list = CustomImageFolder(bg_data_dir,
                                    class_index=1, location_index=4,
                                    transform = get_transforms(bugged_model), limit = num_images)


  # Create a dataloader
  corgi_loader = torch.utils.data.DataLoader(
      class_data_list,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  # Create background dataloader
  bg_loader = torch.utils.data.DataLoader(
      background_data_list,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  # Store image files without transforms if we will be plotting them
  if show_images:
      class_no_transforms_list = CustomImageFolder(class_data_dir,
                                    class_index=class_id, location_index=4,
                                    transform = None, limit = num_images)
      bg_no_transforms_list = CustomImageFolder(bg_data_dir,
                                    class_index=class_id, location_index=4,
                                    transform = None, limit = num_images)

      imgs = load_images_as_np(class_no_transforms_list, num_images)
      imgs_bg = load_images_as_np(bg_no_transforms_list, num_images)


  spurious_scores = []

  concepts_dfs = []

  for model, concept_results in zip([bugged_model, normal_model], [bugged_concept_results, normal_concept_results]):

    if show_images:
      if(model == bugged_model):
        print("Bugged model", "-"*60)
      else:
        print("Normal model", "-"*60)


    # Get the model's FEATURE actviations on corgi images
    X = get_images(corgi_loader, num_images)
    A = get_stacked_activations(model, X, len(X))

    # Get the models' FEATURE activations on background images
    X_bg = get_images(bg_loader, num_images)
    A_bg = get_stacked_activations(model, X_bg, len(X))


    # get CONCEPT activations using GPU
    bg_activations = get_concept_activations_gpu(A_bg, concept_results.concepts)
    activations = get_concept_activations_gpu(A, concept_results.concepts)

    # Calculate differences
    bg_importances = get_concept_importances(model, concept_results.concepts, bg_activations, 1)
    importances = get_concept_importances(model, concept_results.concepts, activations, 1)

    # print(f"\nImportances in background dataset: \n{bg_importances},\nimportances in original dataset: \n{importances}")

    bg_heatmaps = rearrange(bg_activations, '(n h w) r -> n h w r', n=A.shape[0], h=7, w=7)
    heatmaps = rearrange(activations, '(n h w) r -> n h w r', n=A.shape[0], h=7, w=7)

    bg_concepts, concepts_df = filter_background_concepts(heatmaps, bg_heatmaps, concept_results.concepts, num_images, cutoff, method=method, verbose=show_images)
    concepts_dfs.append(concepts_df)
    spurious_scores.append(score(importances, bg_concepts))

    if show_images:
      print(concepts_df)

      print("\n\nComparing Background Concepts")
      for concept_id in bg_concepts:
        compare_concept(concept_id, heatmaps, bg_heatmaps, imgs, imgs_bg, importances)

      print("\n\nComparing Class Concepts")
      for concept_id in range(len(concept_results.concepts)):
        if concept_id not in bg_concepts:
          compare_concept(concept_id, heatmaps, bg_heatmaps, imgs, imgs_bg, importances)
  print(spurious_scores)
  return spurious_scores, concepts_dfs

In [84]:
def single_model_experiment(model,
               class_data_dir, bg_data_dir,
               concept_results, class_idx,
               num_images = 100, show_images = True, method='med-mean', cutoff = 1.0):

  # Create a custom image folder (the Spawrious custom class for datasets)
  class_data_list = CustomImageFolder(class_data_dir,
                                    class_index=1, location_index=4,
                                    transform = get_transforms(model), limit = num_images)
  # Create custom image folder for snow-only data ; these are also labeled as corgi
  background_data_list = CustomImageFolder(bg_data_dir,
                                    class_index=1, location_index=4,
                                    transform = get_transforms(model), limit = num_images)


  # Create a dataloader
  corgi_loader = torch.utils.data.DataLoader(
      class_data_list,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  # Create background dataloader
  bg_loader = torch.utils.data.DataLoader(
      background_data_list,
      batch_size=32,
      shuffle=False,
      num_workers=2,
  )

  # Store image files without transforms if we will be plotting them
  if show_images:
      class_no_transforms_list = CustomImageFolder(class_data_dir,
                                    class_index=class_idx, location_index=4,
                                    transform = None, limit = num_images)
      bg_no_transforms_list = CustomImageFolder(bg_data_dir,
                                    class_index=class_idx, location_index=4,
                                    transform = None, limit = num_images)

      imgs = load_images_as_np(class_no_transforms_list, num_images)
      imgs_bg = load_images_as_np(bg_no_transforms_list, num_images)


  spurious_scores = []

  # Get the model's FEATURE actviations on class images
  X = get_images(corgi_loader, num_images)
  A = get_stacked_activations(model, X, num_images)

  # Get the models' FEATURE activations on background images
  X_bg = get_images(bg_loader, num_images)
  A_bg = get_stacked_activations(model, X_bg, num_images)


  # get CONCEPT activations using GPU
  bg_activations = get_concept_activations_gpu(A_bg, concept_results.concepts)
  activations = get_concept_activations_gpu(A, concept_results.concepts)

  # Calculate differences
  bg_importances = get_concept_importances(model, concept_results.concepts, bg_activations, class_idx)
  importances = get_concept_importances(model, concept_results.concepts, activations, class_idx)

  # print(f"\nImportances in background dataset: \n{bg_importances},\nimportances in original dataset: \n{importances}")

  bg_heatmaps = rearrange(bg_activations, '(n h w) r -> n h w r', n=num_images, h=7, w=7)
  heatmaps = rearrange(activations, '(n h w) r -> n h w r', n=num_images, h=7, w=7)

  bg_concepts, concepts_df = filter_background_concepts(heatmaps, bg_heatmaps, concept_results.concepts, num_images, cutoff, method=method, verbose=show_images)
  spurious_scores.append(score(importances, bg_concepts))

  if show_images:
    print(concepts_df)

    print("\n\nComparing Background Concepts")
    for concept_id in bg_concepts:
      compare_concept(concept_id, heatmaps, bg_heatmaps, imgs, imgs_bg, importances)

    print("\n\nComparing Class Concepts")
    for concept_id in range(len(concept_results.concepts)):
      if concept_id not in bg_concepts:
        compare_concept(concept_id, heatmaps, bg_heatmaps, imgs, imgs_bg, importances)
  print(spurious_scores)
  return spurious_scores, concepts_df

In [154]:
# Helper returns the name of the concept results instance w given inputs
def get_concept_results_name(model_checkpoint, class_idx,  num_images, num_concepts):
  return f"CRAFT_{class_idx}_class_{num_images}_im_{num_concepts}_concepts_{model_checkpoint}"

def create_or_load_concepts(model, model_path_name, class_dir, class_id, num_concepts):
  # Check for existing concepts, generate them if not
  try:
    craft_results_dir = get_concept_results_name(model_path_name, class_id, num_images=300, num_concepts=num_concepts)
    return ConceptResults(craft_results_dir)
  except(FileNotFoundError):
    print("No existing concepts found...")
    print("Generating concepts...")
    craft_results_dir = get_craft_concepts(model, model_path_name, class_dir,
                                          class_id, num_concepts=num_concepts, num_images=300,
                                          results_dir=RESULTS_DIR, visualize=False)
    print("Concept generation complete: ", craft_results_dir)
  return ConceptResults(craft_results_dir)


#6. Run ENTIRE Pipeline (concept generation, spurious test, spurious scores)

In [181]:
def run_pipeline(model_path_name, bg_dataset_dir, normal_dataset_dir, single_model=True):

  all_results = {}

  model = load_model(model_path_name)
  normal_model = load_model(normal_model_name)

  locations = ["desert", "dirt", "jungle", "snow", "mountain"]

  class_list = ["bulldog", "corgi", "dachshund", "labrador"]


  scores = {}
  # Test the spurious-ness of the model; save the results
  scores['bugged'] = [bg_only_test(model, model_path_name, bg_dataset_dir)]
  print("Bugged Model Accuracy on background-only dataset: ", scores['bugged'])
  scores['normal'] = [bg_only_test(normal_model, model_path_name, bg_dataset_dir)]
  print("Normal Model Accuracy on background-only dataset: ", scores['normal'])

  output_dir = os.path.join(EXPERIMENT_OUTPUT_DIR, model_path_name)
  print("Saving Results to: ", output_dir)
  os.makedirs(output_dir, exist_ok=True)

  df = pd.DataFrame(scores)
  df.to_csv(os.path.join(output_dir, "background_only_accuracies.csv"))

  for location in locations:
    print(location)

    # Get class id and name
    class_id = get_class_index(model_path_name, location)
    print(class_id)
    class_name = class_list[class_id]

    print(f"Testing on {class_name} / {location}", "-"*90)
    continue
    # Set dirs
    class_dir = os.path.join(normal_dataset_dir,"0", location, class_name)
    bg_dir = os.path.join(bg_dataset_dir, location+"_only")


    # Dicts for storing results
    results = pd.DataFrame(columns=['method_model', 'spurious_score', 'num_concepts'])
    accuracies = pd.DataFrame(columns=['num_concepts', 'accuracy', 'method'])

    for num_concepts in range(4, 12, 3):
      print(f"  Number of Concepts: {num_concepts}", "-"*50)
      concept_results = create_or_load_concepts(model, model_path_name, class_dir, class_id, num_concepts)
      normal_concept_results = create_or_load_concepts(normal_model, normal_model_name, class_dir, class_id, num_concepts)

      # Gather spurious score
      num_trials = 1

      for method in ['med-mean', 'mean', 'median']:
        print(f"Method: {method}", "-"*20)

        correct = 0

        for i in range (num_trials):

          if single_model:
            scores, concepts_df = single_model_experiment(model, class_dir,
                                bg_dir, concept_results, class_id, num_images=10,
                                show_images = False, method = method, cutoff = 1.0)
            concepts_df.to_csv(os.path.join(output_dir, f"{class_name}_{location}_{num_concepts}_concepts.csv"))

          else:
            scores, concepts_dfs = experiment(model, normal_model, class_dir,
                                bg_dir, concept_results, normal_concept_results,
                                class_id, num_images=10,
                                show_images = False, method = method, cutoff = 1.0)

            concepts_dfs[0].to_csv(os.path.join(output_dir, f"{class_name}_{location}_bugged_{num_concepts}_concepts.csv"))
            concepts_dfs[1].to_csv(os.path.join(output_dir, f"{class_name}_{location}_normal_{num_concepts}_concepts.csv"))

          # Update results df
          new_results = pd.DataFrame({'method_model': [f"{method}_bugged", f"{method}_normal"],
                                      'spurious_score': scores, 'num_concepts': [num_concepts, num_concepts]})
          if results.empty:
            results = new_results
          else:
            results = pd.concat([results, new_results], ignore_index=True)

          # Update prediction accuracy : this measures how well our metric predicts a buggy model
          pred = np.argmax(scores)
          correct += (pred == 0)


        new_accuracy = pd.DataFrame({'method': method, 'accuracy': [correct / num_trials], 'num_concepts': [num_concepts]})
        # Update accuracy df
        if accuracies.empty:
            accuracies = new_accuracy
        else:
            accuracies = pd.concat([accuracies, new_accuracy], ignore_index=True)

    # Save spurious scores
    results.to_csv(os.path.join(output_dir, f"{class_name}_{location}_Spurious_scores.csv"))
    # Save accuracies
    accuracies.to_csv(os.path.join(output_dir, f"{class_name}_{location}_Spurious_Detection_Accuracies.csv"))

  return output_dir

In [184]:
output_dir = run_pipeline(bugged_model_name, "/content/data/background_only_dataset", "/content/data/spawrious224", single_model=False)

<ipython-input-9-a9afaae834bd>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load(os.path.join(models_dir, checkpoint_path))


Loc:  /content/data/background_only_dataset/jungle_only cls:  1
Loc:  /content/data/background_only_dataset/dirt_only cls:  3


Bugged Model Accuracy on background-only dataset:  [27.5]
Loc:  /content/data/background_only_dataset/jungle_only cls:  1
Loc:  /content/data/background_only_dataset/dirt_only cls:  3


Normal Model Accuracy on background-only dataset:  [45.0]
Saving Results to:  /content/drive/MyDrive/CS2822_Final_Project/Final_Results/o2o_medium_resnet18.a1-e=2-lr=0.01_limit=20.pt
desert
-1
Testing on labrador / desert ------------------------------------------------------------------------------------------
dirt
3
Testing on labrador / dirt ------------------------------------------------------------------------------------------
jungle
1
Testing on corgi / jungle ------------------------------------------------------------------------------------------
snow
-1
Testing on labrador / snow ------------------------------------------------------------------------------------------
mountain
0
Testing on bulldog / mountain ------------------------------------------------------------------------------------------
